# Notebook configuration

In [68]:
# Configuration
shapefile_dir_path = '/home/bertrand/Documents/Projet Indus/WazeTT/data/input/bd_topo/'
shapefile_name = 'ZONE_VEGETATION.shp'
shapefile_path = shapefile_dir_path + shapefile_name

tiles_dir_path = '../data/input/bd_alti/'
tiles_name = 'dalles.shp'
tiles_path = tiles_dir_path + tiles_name

ascii_file_dir_path = '../data/input/bd_alti/'
ascii_file_name = 'BDALTIV2_25M_FXX_0825_6550_MNT_LAMB93_IGN69.asc'
ascii_file_path = ascii_file_dir_path + ascii_file_name

# Load the altitudes ascii file into an AltiData object

In [69]:
from alti_data import AltiData
alti_data = AltiData(ascii_file_path)

# Deduce its Bounding Box coordinates in Lamb93 (xll, yll)

In [70]:
alti_data_extreme_bb_coordinates_lamb93 = alti_data.calc_bb_extreme_coordinates(format='lamb93')
print(alti_data_extreme_bb_coordinates_lamb93)

alti_data_bb_angle_coordinates_lamb93 = alti_data.calc_bb_angle_coordinates(format='lamb93')
print(alti_data_bb_angle_coordinates_lamb93)

{'xmin': 824987.5, 'xmax': 849987.5, 'ymin': 6500012.5, 'ymax': 6525012.5}
{'nw': (824987.5, 6525012.5), 'ne': (849987.5, 6525012.5), 'se': (849987.5, 6500012.5), 'sw': (824987.5, 6500012.5)}


# Create ASCII table Bounding Box polygon in Lamb93

In [71]:
ascii_table_bb_polygon_lamb93 = alti_data.calc_bb_polygon(format='lamb93')
print(ascii_table_bb_polygon_lamb93)

POLYGON ((824987.5 6525012.5, 849987.5 6525012.5, 849987.5 6500012.5, 824987.5 6500012.5, 824987.5 6525012.5))


# Display a Polygon on world map

In [72]:
import geopandas as gpd
from pyproj import CRS, Transformer
import folium
import json
import shapely

In [73]:
alti_data_bb_polygon = json.dumps(shapely.geometry.mapping(alti_data.calc_bb_polygon(format='wgs84')),indent=4)

m = folium.Map(
    location=[45.6, 4.9],
    tiles="cartodbpositron",
    zoom_start=10,
)

folium.GeoJson(alti_data_bb_polygon, name="Bounding Box").add_to(m)

# Add the possibility to the user to display the layers or not (The layers are displayed by default.)
folium.LayerControl().add_to(m)

m

# Load the field shapefile

In [77]:
# Conversion of a pair of files (.shp + .shx) into a list of POLYGON included into the ASCII table Bounding Box
import geopandas as gpd

bbox_extreme_coordinates = (
                            alti_data_extreme_bb_coordinates_lamb93['xmin'],
                            alti_data_extreme_bb_coordinates_lamb93['xmax'],
                            alti_data_extreme_bb_coordinates_lamb93['ymin'],
                            alti_data_extreme_bb_coordinates_lamb93['ymax'],
)
gdf = gpd.read_file(
    shapefile_path,
    bbox=bbox_extreme_coordinates,
)

shapefile_polygones_lamb93 = gdf['geometry']
print(shapefile_polygones_lamb93)

0       POLYGON ((907748.000 6520508.100, 907742.900 6...
1       POLYGON ((910072.900 6523975.600, 910068.900 6...
2       POLYGON ((896896.000 6524342.400, 896913.800 6...
3       POLYGON ((896004.200 6520785.000, 895999.000 6...
4       POLYGON ((912960.900 6516887.800, 912958.500 6...
                              ...                        
9155    POLYGON ((913716.100 6520221.600, 913726.400 6...
9156    POLYGON ((913955.300 6520072.300, 913957.700 6...
9157    POLYGON ((910987.700 6520097.700, 911065.400 6...
9158    POLYGON ((915525.700 6520592.100, 915586.500 6...
9159    POLYGON ((915283.600 6523628.200, 915286.400 6...
Name: geometry, Length: 9160, dtype: geometry


# Plot the first polygon of the shapefile

In [78]:
from coordinates_utils import wgs84_epsg_code

gdf_wgs84 = gdf.to_crs("EPSG:"+str(wgs84_epsg_code))
shapefile_polygones_wgs84 = gdf_wgs84['geometry']

m = folium.Map(
    location=[45.6, 4.9],
    tiles="cartodbpositron",
    zoom_start=10,
)

folium.GeoJson(shapefile_polygones_wgs84[0:10], name="Bounding Box").add_to(m)

# Add the possibility to the user to display the layers or not (The layers are displayed by default.)
folium.LayerControl().add_to(m)

m

# Creation of the field table to project the shapefiles onto

In [79]:
# Initializes the empty field 2D table
field_table = [['default_field' for col in range(alti_data.ncols)] for row in range(alti_data.nrows)]
print('Number of rows : {}'.format(len(field_table[0])))
print('Number of columns : {}'.format(len(field_table)))
print('Cells content : {}'.format(field_table[0][0]))

Number of rows : 1000
Number of columns : 1000
Cells content : default_field


# Calculate polygons bounding boxes

In [80]:
polygon_bb_bounds = []
for polygon in shapefile_polygones_lamb93:
    polygon_bb_bounds.append(polygon.bounds)

from coordinates_utils import bb_bounds_to_coord
polygon_bb_coordinates = bb_bounds_to_coord(polygon_bb_bounds)
print(polygon_bb_coordinates)

{'nw': ((907737.6, 6520504.1, 907748.2, 6520518.3), (895994.9, 6520783.4, 896007.0, 6520796.3)), 'ne': ((894041.9, 6520992.0, 897677.2, 6524369.3), (895994.9, 6520783.4, 896007.0, 6520796.3)), 'se': ((894041.9, 6520992.0, 897677.2, 6524369.3), (910058.3, 6523969.8, 910072.9, 6523980.8)), 'sw': ((907737.6, 6520504.1, 907748.2, 6520518.3), (910058.3, 6523969.8, 910072.9, 6523980.8))}


# Plot the 10 first polygons (blue) and their BB (red)?

# (Not useful anymore)
Convert shapefile polygons coordinates from WGS84 to Lamb93

In [41]:
# from pyproj import Transformer
# lamb93_epsg_code = "2154" # The Lamb93 coordinates system -> https://epsg.io/2154
# wgs84_epsg_code = "4326" # The WGS84 coordinates system -> https://epsg.io/4326
# transformer = Transformer.from_crs("epsg:"+wgs84_epsg_code, "epsg:"+lamb93_epsg_code)

# for shapefile_polygon_wgs84 in shapefile_polygones_wgs84:
#     ascii_table_nw_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_nw_bound_coord_lamb93[0],
#                                                                 ascii_table_nw_bound_coord_lamb93[1]
#                                                             )
#     ascii_table_ne_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_ne_bound_coord_lamb93[0],
#                                                                 ascii_table_ne_bound_coord_lamb93[1]
#                                                             )
#     ascii_table_se_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_se_bound_coord_lamb93[0],
#                                                                 ascii_table_se_bound_coord_lamb93[1]
#                                                             )
#     ascii_table_sw_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_sw_bound_coord_lamb93[0],
#                                                                 ascii_table_sw_bound_coord_lamb93[1]
#                                                             )

# print('ascii_table_nw_bound_coord_wgs84 : {}'.format(ascii_table_nw_bound_coord_wgs84))
# print('ascii_table_ne_bound_coord_wgs84 : {}'.format(ascii_table_ne_bound_coord_wgs84))
# print('ascii_table_se_bound_coord_wgs84 : {}'.format(ascii_table_se_bound_coord_wgs84))
# print('ascii_table_sw_bound_coord_wgs84 : {}'.format(ascii_table_sw_bound_coord_wgs84))

# # TODO : visualize these data with follium

ascii_table_nw_bound_coord_wgs84 : (inf, inf)
ascii_table_ne_bound_coord_wgs84 : (inf, inf)
ascii_table_se_bound_coord_wgs84 : (inf, inf)
ascii_table_sw_bound_coord_wgs84 : (inf, inf)


# (Not useful anymore)
Project the Polygons extracted from the shapefile onto the 2D ASCII table (William)

In [9]:
# from shapely.geometry import Polygon

# projected_polygons = []
# for shapefile_polygon_wgs84 in shapefile_polygones_wgs84:
#     projected_polygon = ascii_table_bb_polygon_wgs84.intersection(shapefile_polygon_wgs84)
#     if projected_polygon.__str__() != 'POLYGON EMPTY':
#         print(projected_polygon)
#         projected_polygons.append(projected_polygon)

TypeError: 'Polygon' object is not iterable

# Merge all the tiles information into a single data structure (Jérémy)

In [ ]:
merge_all = []
Dalle1 = {}
Dalle1['alti']=ascii_table
Dalle1['milieu1']=[]
Dalle1['milieu2']=[]

merge_all.append(Dalle1)

# (Optionnal) Get shapefile BoundingBox coordinates
WGS84 coordinates system (lat, long)

In [36]:
# xml_path = gmd:MD_Metadata/gmd:identificationInfo/gmd:MD_DataIdentification/gmd:extent/gmd:EX_Extent/gmd:geographicElement/gmd:EX_GeographicBoundingBox
import xml.etree.ElementTree as ET
metadata_file_name = 'metadonnees.xml'
tree = ET.parse(shapefile_dir_path + metadata_file_name)
root = tree.getroot()

namespaces = {
                'gmd': '{http://www.isotc211.org/2005/gmd}',
                'gco': '{http://www.isotc211.org/2005/gco}'
            } # add more as needed

EX_GeographicBoundingBox_element = list(root.iter(namespaces['gmd']+'EX_GeographicBoundingBox'))[0]

west_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'westBoundLongitude'))[0]
west_bound_longitude_decimal_element = list(west_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
west_bound_longitude_decimal_value = list(west_bound_longitude_decimal_element.itertext())[0]
print('west_bound_longitude_decimal_value : {}'.format(west_bound_longitude_decimal_value))

east_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'eastBoundLongitude'))[0]
east_bound_longitude_decimal_element = list(east_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
east_bound_longitude_decimal_value = list(east_bound_longitude_decimal_element.itertext())[0]
print('east_bound_longitude_decimal_value : {}'.format(east_bound_longitude_decimal_value))

south_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'southBoundLatitude'))[0]
south_bound_latitude_decimal_element = list(south_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
south_bound_latitude_decimal_value = list(south_bound_latitude_decimal_element.itertext())[0]
print('south_bound_latitude_decimal_value : {}'.format(south_bound_latitude_decimal_value))

north_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'northBoundLatitude'))[0]
north_bound_latitude_decimal_element = list(north_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
north_bound_latitude_decimal_value = list(north_bound_latitude_decimal_element.itertext())[0]
print('north_bound_latitude_decimal_value : {}'.format(north_bound_latitude_decimal_value))

west_bound_longitude_decimal_value : 2.0629
east_bound_longitude_decimal_value : 7.1856
south_bound_latitude_decimal_value : 44.1155
north_bound_latitude_decimal_value : 46.8043
